In [1]:
# pip install -U langchain-community tavily-python
# pip install --upgrade --quiet  langchain-google-genai

In [2]:
import json
import os
from langchain_fireworks import ChatFireworks
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt.tool_node import ToolNode

In [3]:
# Define the function that determines whether to continue or not
def should_continue(messages):
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return END
    else:
        return "action"

In [8]:
# Define a new graph
workflow = MessageGraph()

tools = [TavilySearchResults(max_results=1)]
model = ChatFireworks(model="accounts/fireworks/models/firefunction-v1")
workflow.add_node("agent", model)
workflow.add_node("action", ToolNode(tools))

workflow.set_entry_point("agent")

# Conditional agent -> action OR agent -> END
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

# Always transition `action` -> `agent`
workflow.add_edge("action", "agent")

memory = SqliteSaver.from_conn_string(":memory:") # Here we only save in-memory

# Setting the interrupt means that any time an action is called, the machine will stop
app = workflow.compile(checkpointer=memory, interrupt_before=["action"])

In [9]:
# Run the graph
thread = {"configurable": {"thread_id": "4"}}
for event in app.stream("what is the weather in sf currently", thread, stream_mode="values"):
    event[-1].pretty_print()

================================ Human Message =================================

what is the weather in sf currently
================================== Ai Message ==================================

I'm sorry, but I don't have the ability to fetch real-time weather information. My current capabilities are limited to the functions provided to me.
